In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))
from spider2_utils import load_csv_database

-setup-

In [ ]:
import pandas as pd
_database = load_csv_database("education_business", rows_limit=-1)
university_faculty = _database["university_faculty"]

# Question
Which university faculty members' salaries are closest to the average salary for their respective ranks? Please provide the ranks, first names, last names, and salaries.university

###  Step 1: User intent — Compute average salary per faculty rank

In [ ]:
avg_salaries = university_faculty.groupby('FacRank', as_index=False)['FacSalary'].mean()
avg_salaries.rename(columns={'FacRank': 'FacRank', 'FacSalary': 'AvSalary'}, inplace=True)

###  Step 2: User intent — Join average salaries back to the original table

In [ ]:
salary_diffs = university_faculty.merge(avg_salaries, left_on='FacRank', right_on='FacRank')

###  Step 3: User intent — Compute absolute salary difference from rank average

In [ ]:
salary_diffs['Diff'] = (salary_diffs['FacSalary'] - salary_diffs['AvSalary']).abs()

###  Step 4: User intent — Compute minimum salary difference per rank

In [ ]:
min_diffs = salary_diffs.groupby('FacRank', as_index=False)['Diff'].min()
min_diffs.rename(columns={'Diff': 'MinDiff'}, inplace=True)

###  Step 5: User intent — Join back to find faculty with the minimum salary difference per rank

In [ ]:
final_result = salary_diffs.merge(min_diffs, left_on=['FacRank', 'Diff'], right_on=['FacRank', 'MinDiff'])
final_result = final_result[['FacRank', 'FacFirstName', 'FacLastName', 'FacSalary']]
final_result